Versão aprimorada do Software de Otimizaçção de Rotas!

In [ ]:
# Instalação da biblioteca DEAP
!pip install deap

In [ ]:

import random
import numpy as np
from deap import base, creator, tools, algorithms
import time

# Constantes
TAXA_MUTACAO_INICIAL = 0.1
ELITISMO_SIZE = 5
MAX_GERACOES_SEM_MELHORIA = 15
MAX_GERACOES_BASE = 500
TEMPO_MAXIMO = 60  # segundos
TAMANHO_POPULACAO_BASE = 100

# Pontos de entrega e matriz de distâncias (exemplo)
PONTOS = {
    1: "Almoxarifado",
    2: "Biblioteca",
    3: "Bloco 1 de Aulas",
    4: "Bloco 2 de Aulas",
    5: "Bloco administrativo",
    6: "Bloco 1 de Professores",
    7: "Bloco 2 de Professores",
    8: "Estacionamento",
    9: "Garagem",
    10: "Ginásio",
    11: "Laboratórios 1",
    12: "Laboratórios 2 ",
    13: "Memorial Paulo Freire",
    14: "Praça das Flores",
    15: "Restaurante Universitário",
    16:"Condomínio Canaã",
    17:"Sala do Prof Maristélio Costa",
}

# Matriz de distâncias atualizada (em metros)
DISTANCIAS = np.array([
    [0, 350, 550, 290, 500, 140, 290, 750, 2, 120, 350, 400, 450, 170, 110, 1200, 1400],  # Almoxarifado
    [350, 0, 400, 190, 190, 210, 190, 400, 350, 350, 150, 240, 350, 190, 250, 900, 1100],  # Biblioteca
    [550, 400, 0, 400, 400, 400, 400, 600, 550, 550, 260, 160, 600, 400, 450, 1100, 1300],  # Bloco 1 de Aulas
    [290, 190, 400, 0, 350, 150, 130, 210, 290, 290, 200, 230, 190, 120, 180, 1100, 1300],  # Bloco 2 de Aulas
    [500, 190, 400, 350, 0, 350, 350, 550, 500, 500, 130, 230, 500, 350, 400, 750, 950],  # Bloco administrativo
    [140, 210, 400, 150, 350, 0, 150, 350, 150, 140, 230, 260, 300, 27, 34, 1100, 1300],  # Bloco 1 de Professores
    [290, 190, 400, 130, 350, 150, 0, 200, 290, 280, 210, 240, 170, 120, 180, 1100, 1300],  # Bloco 2 de Professores
    [500, 400, 600, 210, 550, 350, 200, 0, 500, 500, 400, 450, 56, 300, 400, 1300, 1500],  # Estacionamento
    [2, 350, 550, 290, 500, 150, 290, 500, 0, 120, 400, 400, 450, 170, 110, 1200, 1400],  # Garagem
    [120, 350, 550, 290, 500, 140, 280, 500, 120, 0, 350, 400, 450, 170, 110, 1200, 1400],  # Ginásio
    [350, 150, 260, 200, 130, 230, 210, 400, 400, 350, 0, 95, 400, 200, 270, 850, 1100],  # Laboratórios 1
    [400, 240, 160, 230, 230, 260, 240, 450, 400, 400, 95, 0, 400, 240, 300, 1000, 1200],  # Laboratórios 2
    [450, 350, 600, 190, 500, 300, 170, 56, 450, 450, 400, 400, 0, 290, 350, 1200, 1400],  # Memorial Paulo Freire
    [170, 190, 400, 120, 350, 27, 120, 300, 170, 170, 200, 240, 290, 0, 61, 1100, 1300],  # Praça das Flores
    [110, 250, 450, 180, 400, 34, 180, 400, 110, 110, 270, 300, 350, 61, 0, 1100, 1300],  # Restaurante Universitário
    [1200, 900, 1100, 1100, 750, 1100, 1100, 1300, 1200, 1200, 850,1000, 1200, 1100, 1100, 0, 210], # Condomínio Canaã
    [1400, 1100, 1300, 1300, 950, 1300, 1300, 1500, 1400, 1400, 1100, 1200, 1400, 1300, 1300, 210, 0], # Sala do Prof Maristelio Costa
])

# Função para calcular a distância total de um percurso
def calcular_distancia_total(individuo, distancias):
    distancia_total = 0
    for i in range(len(individuo) - 1):
        idx1 = individuo[i]
        idx2 = individuo[i + 1]
        distancia_total += distancias[idx1][idx2]
    # Adiciona a distância de retorno ao ponto inicial
    distancia_total += distancias[individuo[-1]][individuo[0]]
    return distancia_total

# Função de avaliação do fitness (distância total)
def avaliar(individuo):
    distancia_total = calcular_distancia_total(individuo, DISTANCIAS)
    return (distancia_total,)

# Definição dos tipos do DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Função para exibir a matriz de distâncias
def exibir_matriz_distancias(pontos_selecionados):
    print("\nMatriz de Distâncias (em metros):")
    num_pontos = len(pontos_selecionados)
    tamanho_celula = 10

    # Cabeçalho da tabela
    print("┌" + "─" * tamanho_celula + ("┬" + "─" * tamanho_celula) * num_pontos + "┐")
    print("│" + f"{'':^{tamanho_celula}}", end="")
    for ponto in pontos_selecionados:
        print(f"│{ponto[:tamanho_celula]:^{tamanho_celula}}", end="")
    print("│")
    print("├" + "─" * tamanho_celula + ("┼" + "─" * tamanho_celula) * num_pontos + "┤")

    # Linhas da matriz
    for i, ponto1 in enumerate(pontos_selecionados):
        print(f"│{ponto1[:tamanho_celula]:^{tamanho_celula}}", end="")
        for j, ponto2 in enumerate(pontos_selecionados):
            # Obtém os números dos pontos (chaves do dicionário PONTOS)
            idx1 = list(PONTOS.keys())[list(PONTOS.values()).index(ponto1)]
            idx2 = list(PONTOS.keys())[list(PONTOS.values()).index(ponto2)]
            # Acessa a matriz DISTANCIAS usando os índices corretos
            distancia = DISTANCIAS[idx1 - 1][idx2 - 1]  # Subtrai 1 porque a matriz começa em 0
            print(f"│{distancia:^{tamanho_celula}}", end="")
        print("│")
        if i < num_pontos - 1:
            print("├" + "─" * tamanho_celula + ("┼" + "─" * tamanho_celula) * num_pontos + "┤")

    print("└" + "─" * tamanho_celula + ("┴" + "─" * tamanho_celula) * num_pontos + "┘")

# Função principal do algoritmo genético
def algoritmo_genetico(pontos_selecionados, taxa_mutacao_inicial=TAXA_MUTACAO_INICIAL,
                      elitismo_size=ELITISMO_SIZE, max_geracoes_sem_melhoria=MAX_GERACOES_SEM_MELHORIA,
                      tempo_maximo=TEMPO_MAXIMO):
    num_pontos = len(pontos_selecionados)
    tamanho_populacao = min(TAMANHO_POPULACAO_BASE, 10 * num_pontos)
    num_geracoes = MAX_GERACOES_BASE

    toolbox = base.Toolbox()
    toolbox.register("indices", random.sample, range(num_pontos), num_pontos)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, tamanho_populacao)
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=taxa_mutacao_inicial)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", avaliar)

    pop = toolbox.population()
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    best_individuals = tools.selBest(pop, elitismo_size)
    melhor_fitness = best_individuals[0].fitness.values[0]
    geracoes_sem_melhoria = 0

    inicio = time.time()

    for g in range(num_geracoes):
        if time.time() - inicio > tempo_maximo:
            print(f"Tempo máximo de execução ({tempo_maximo}s) atingido.")
            break

        offspring = toolbox.select(pop, len(pop))  # Seleciona todos os indivíduos
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < taxa_mutacao_inicial:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring + best_individuals
        best_individuals = tools.selBest(pop, elitismo_size)

        novo_melhor_fitness = best_individuals[0].fitness.values[0]
        if novo_melhor_fitness < melhor_fitness:
            melhor_fitness = novo_melhor_fitness
            geracoes_sem_melhoria = 0
        else:
            geracoes_sem_melhoria += 1

        if geracoes_sem_melhoria >= max_geracoes_sem_melhoria:
            print(f"Convergência atingida após {g + 1} gerações.")
            break

    return tools.selBest(pop, 1)[0]

# Função para perguntar ao usuário se deseja tentar novamente
def perguntar_tentar_novamente():
    while True:
        resposta = input("\nDeseja tentar novamente? (s/n): ").strip().lower()
        if resposta in ['s', 'n']:
            return resposta == 's'
        else:
            print("Por favor, digite 's' para sim ou 'n' para não.")

# Função principal
def main():
    while True:
        print("Bem-vindo ao Software de Otimização de Rotas com Algoritmos Genéticos!")
        print("Lista de Locais de Entrega Disponíveis:")
        for idx, ponto in PONTOS.items():
            print(f"{idx}: {ponto}")

        # Seleção de pontos pelo usuário
        while True:
            try:
                entrada = input("Informe os números dos locais de entrega desejados, separados por vírgula: ")
                pontos_selecionados = [int(idx) for idx in entrada.split(',')]

                # Verifica se todos os índices são válidos e únicos
                if all(idx in PONTOS for idx in pontos_selecionados) and len(pontos_selecionados) >= 2:
                    break
                else:
                    print("Erro: Entrada inválida. Certifique-se de que todos os números são válidos e há pelo menos dois locais selecionados.")
            except ValueError:
                print("Erro: Entrada inválida. Digite apenas números separados por vírgula.")

        # Seleção do ponto de partida
        while True:
            try:
                ponto_partida = int(input("Informe o número do ponto de partida: "))
                if ponto_partida in pontos_selecionados:
                    break
                else:
                    print("Erro: O ponto de partida deve ser um dos pontos selecionados.")
            except ValueError:
                print("Erro: Entrada inválida. Digite apenas números.")

        # Converte os índices para os nomes dos pontos
        pontos_selecionados = [PONTOS[idx] for idx in pontos_selecionados]

        # Exibir matriz de distâncias
        exibir_matriz_distancias(pontos_selecionados)

        # Execução do algoritmo genético
        melhor_rota_indices = algoritmo_genetico(pontos_selecionados)
        melhor_rota = [pontos_selecionados[i] for i in melhor_rota_indices]

        # Exibição dos resultados
        print("\nMelhor Trajeto Calculado:")

        # Reorganiza a rota para começar no ponto de partida escolhido pelo usuário
        ponto_partida_nome = PONTOS[ponto_partida]  # Converte o índice do ponto de partida para o nome
        indice_partida = melhor_rota.index(ponto_partida_nome)  # Encontra o índice do ponto de partida na rota
        melhor_rota_ordenada = melhor_rota[indice_partida:] + melhor_rota[:indice_partida]  # Reorganiza a rota

        # Exibe a rota em ordem sequencial
        distancia_total = 0
        for i in range(len(melhor_rota_ordenada)):
            ponto_atual = melhor_rota_ordenada[i]
            proximo_ponto = melhor_rota_ordenada[(i + 1) % len(melhor_rota_ordenada)]
            # Obtém os índices corretos dos pontos na matriz DISTANCIAS
            idx1 = list(PONTOS.keys())[list(PONTOS.values()).index(ponto_atual)]
            idx2 = list(PONTOS.keys())[list(PONTOS.values()).index(proximo_ponto)]
            distancia = DISTANCIAS[idx1 - 1][idx2 - 1]  # Subtrai 1 porque a matriz começa em 0
            distancia_total += distancia
            print(f"{i + 1}. {ponto_atual} -> {proximo_ponto} ({distancia} metros)")
        print(f"\nDistância total Percorrida: {distancia_total} metros")

        # Pergunta ao usuário se deseja tentar novamente
        if not perguntar_tentar_novamente():
            print("Obrigado por usar o Software de Otimização de Rotas. Até logo!")
            break

# Execução do programa
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")